In [1]:
# Dependencies
import requests
import json
import os
import gmaps
from pandas.io.json import json_normalize
import numpy as np
import pandas as pd
import time

# Google  API key
from config import gkey

gmaps.configure(api_key=gkey)

In [2]:
# Getting a list of Nightclubs in San Francisco
# We want to demostrate the use of APIs
target_coordinates = "37.7749,-122.4194"
target_search = "Nightclub"
target_radius = 50000
target_type = "nightclub"

# Setting up parameters
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [3]:
# convert response to json
sf_clubs = response.json()

# Print the json (pretty printed)
print(json.dumps(sf_clubs, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "next_page_token": "CrQCKwEAAPOXgVrXjdn_Mdc1nmgLwSd6MlpAb_G_k3hWExoxmlXRpXIEr7rQOol88uM2Y_fctOnQ-scEG0PIQ__V32rzFPWnVBkgOaaFBTeRDsTc0vfbcRSybYnazQG-k9YgRStGHae9rGUx5hdsCkgwfxUZ4zvy1wujmAT0i6tFZ-NS1e6wYUoRBxSLXl7DZQM-odmQu2pd2U26Q63PhfPxzrVTJLPCgKMtRnoB2bvL4Nx_PvOMT59WWDVjeqJzAEWpFPsV6rzrpx-uj_N28ti9SFC-6ME9Iba_Ct_dZ6ixPtqZuqKDDu-0ov8fwwUUhqUBxBbCWPrPR9mUjgE1mS3CamOjg9-lx0BqYApVH2uglY8kz81ADGL8rcsxcfkZwfJ3OA6oS8SeZ7zIqNL4SNuUEZ8INWISECRgY5sop8ZW1I4y1T5djOUaFNLd7tFPqDcAUz9y2carzCmSk-rJ",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 37.3768037,
                    "lng": -122.0298038
                },
                "viewport": {
                    "northeast": {
                        "lat": 37.37818862989272,
                        "lng": -122.0286148701073
                    },
                    "southwest": {
                        "lat": 37.37548897010728,
                    

In [4]:
print(len(sf_clubs["results"]))

20


In [5]:
name=[]
vicinity =[]
lat = []
lng =[]
i= 0
while i < 20:
        name.append(sf_clubs["results"][i]["name"])
        vicinity.append((sf_clubs["results"][i]["vicinity"]))
        lat.append(sf_clubs["results"][i]['geometry']['location']['lat'])
        lng.append(sf_clubs["results"][i]['geometry']['location']['lng'])
        i+=1

clubs_dict = {
    "Club Name": name
    , "Address": vicinity
    , "Latitude": lat
    , "Longitude": lng
}
clubs_data = pd.DataFrame(clubs_dict)
clubs_data['Latitude'] = clubs_data['Latitude'].apply(lambda x: round(x, 3))
clubs_data['Longitude'] = clubs_data['Longitude'].apply(lambda x: round(x, 3))
clubs_data.head()

,Club Name,Address,Latitude,Longitude
0,Pure Nightclub,"146 S Murphy Ave, Sunnyvale",37.377,-122.030
1,HUE,"447 Broadway, San Francisco",37.798,-122.405
2,Origin,"1538 Fillmore St, San Francisco",37.784,-122.433
3,The Grand,"520 4th St, San Francisco",37.779,-122.398
4,F8,"1192 Folsom St, San Francisco",37.775,-122.410


In [6]:
crime=pd.read_csv("sf_crime.csv")

In [7]:
#Drop unncessary columns
new_crime = crime[['Category', 'Descript', 'X', 'Y']]
new_crime.columns = ['Category', 'Description', 'Longitude', 'Latitude']
new_crime['Longitude'] = new_crime['Longitude'].apply(lambda x: round(x, 3))
new_crime['Latitude'] = new_crime['Latitude'].apply(lambda x: round(x, 3))
new_crime.head()

C:\Users\hjp28\anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\hjp28\anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Category,Description,Longitude,Latitude
0,WEAPON LAWS,POSS OF PROHIBITED WEAPON,-122.403,37.775
1,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",-122.403,37.775
2,WARRANTS,WARRANT ARREST,-122.389,37.730
3,NON-CRIMINAL,LOST PROPERTY,-122.413,37.786
4,NON-CRIMINAL,LOST PROPERTY,-122.420,37.765


In [8]:
combo_df = pd.merge(clubs_data, new_crime, how='left', left_on=['Latitude', 'Longitude'],right_on=['Latitude','Longitude'])
combo_df

,Club Name,Address,Latitude,Longitude,Category,Description
0,Pure Nightclub,"146 S Murphy Ave, Sunnyvale",37.377,-122.030,NaN,NaN
1,HUE,"447 Broadway, San Francisco",37.798,-122.405,ASSAULT,"BATTERY, FORMER SPOUSE OR DATING RELATIONSHIP"
2,HUE,"447 Broadway, San Francisco",37.798,-122.405,WARRANTS,ENROUTE TO ADULT AUTHORITY
3,HUE,"447 Broadway, San Francisco",37.798,-122.405,LARCENY/THEFT,PETTY THEFT FROM A BUILDING
4,HUE,"447 Broadway, San Francisco",37.798,-122.405,OTHER OFFENSES,RESISTING ARREST
5,HUE,"447 Broadway, San Francisco",37.798,-122.405,SUSPICIOUS OCC,INVESTIGATIVE DETENTION
6,HUE,"447 Broadway, San Francisco",37.798,-122.405,NON-CRIMINAL,LOST PROPERTY
7,HUE,"447 Broadway, San Francisco",37.798,-122.405,DRUG/NARCOTIC,SALE OF COCAINE
8,HUE,"447 Broadway, San Francisco",37.798,-122.405,DRUG/NARCOTIC,SALE OF MARIJUANA
9,HUE,"447 Broadway, San Francisco",37.798,-122.405,DRUG/NARCOTIC,POSSESSION OF COCAINE FOR SALES


In [9]:
combo_df.to_csv('club_crime.csv', index=False)